In [21]:
from vantage6.client import Client

In [134]:
class NewClient(Client):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.use_master_container = False
        self.collaboration = None
        self.image = None
        
    def wait_and_collect_results(client, task_id):
        task = client.get_task(task_id)
        while not task.get("complete"):
            task = client.get_task(task_id)
            info("Waiting for results")
            time.sleep(1)

        info("Obtaining results")
        results = client.get_results(task_id=task.get("id"))
        return results
    
    def call(self, method, input_, master=False):
        assert self.collaboration
        assert self.image
        pass
    
    def get_collaboration_nodes(self, collaboration_id=None, fields=None):
        col_id = collaboration_id or self.collaboration
        assert col_id
        res = client.request(f'collaboration/{col_id}/node')
        if fields:
            return [self._filter_dict(adict, fields) for adict in res]
        else:
            return res
    
    def get_my_nodes(self, fields=None):
        res = client.request(f'organization/{self.whoami.organization_id}/node')
        if fields:
            return [self._filter_dict(adict, fields) for adict in res]
        else:
            return res
        
    def get_server_version(self):
        return client.request('version')
    
    def get_my_organization(self, fields=None):
        res = client.request(f'organization/{self.whoami.organization_id}')
        if fields:
            return self._filter_dict(res, fields)
        else:
            return res
    
    def get_my_collaborations(self, fields=None):
        res = client.request(f'organization/{self.whoami.organization_id}/collaboration')
        if fields:
            return [self._filter_dict(adict, fields) for adict in res]
        else:
            return res
        
    def get_my_details(self, fields=None):
        res = client.request(f'user/{self.whoami.id_}')
        if fields:
            return self._filter_dict(res, fields)
        else:
            return res
    
    @staticmethod
    def _filter_dict(dict_, keys):
        return {k:dict_[k] for k in keys if k in dict_}
   
        
    

In [135]:
client = NewClient("https://trolltunga.vantage6.ai", 443, "")
client.authenticate("frank@iknl.nl", "dude123")

In [136]:
client.setup_encryption(None)

In [137]:
client.get_server_version()

{'version': '1.2.0'}

In [138]:
client.get_my_collaborations(('id','name'))

[{'id': 2, 'name': 'IKNL-Palga'},
 {'id': 4, 'name': 'PREDICT'},
 {'id': 6, 'name': 'JRC'},
 {'id': 8, 'name': 'IKNL - TCR'},
 {'id': 10, 'name': 'IKNL-INT-ISS'}]

In [139]:
client.get_my_organization(('id',))

{'id': 4}

In [140]:
client.get_my_details(('id', 'username','email',))

{'id': 8, 'username': 'frank@iknl.nl', 'email': 'f.martin@iknl.nl'}

In [141]:
client.get_collaboration_nodes(2, fields=('id', 'name', 'status'))

[{'id': 14, 'name': 'Palga IKNL-palga', 'status': 'offline'},
 {'id': 13, 'name': 'IKNL IKNL-palga', 'status': 'offline'}]

In [147]:
client.get_my_nodes(('id','name',))

[{'id': 13, 'name': 'IKNL IKNL-palga'},
 {'id': 38, 'name': 'IKNL - INT IKNL node'},
 {'id': 42, 'name': 'IKNL - JRC Node'},
 {'id': 35, 'name': 'TCR - IKNL IKNL node'}]

In [149]:
client.get_my_nodes(('ip',))

[{'ip': '127.0.0.1'},
 {'ip': '172.17.0.1'},
 {'ip': '172.17.0.1'},
 {'ip': '172.17.0.1'}]